In [ ]:
%run -i 'commons.ipynb'
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train, x_test = x_train / 255.0, x_test / 255.0

plot_mnist_sample(x_train, y_train)

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10)
model.evaluate(x_test,  y_test, verbose=2)

In [ ]:
%env S3_USE_HTTPS=0 #set to 0 for default minio installs
%env S3_ENDPOINT=minio-service.kubeflow:9000
%env AWS_ENDPOINT_URL=http://minio-service.kubeflow:9000
%env AWS_ACCESS_KEY_ID=minio
%env AWS_SECRET_ACCESS_KEY=minio123

In [ ]:
import boto3
import tensorflow as tf

export_bucket = "models"
export_path = "s3://{}/mnist_export/2".format(export_bucket)

# Checking existing buckets and creeating an export bucket if it doesn't exist
s3 = boto3.client('s3',
                  endpoint_url='http://minio-service.kubeflow:9000',
                  aws_access_key_id='minio',
                  aws_secret_access_key='minio123')

response = s3.list_buckets()
export_bucket_exists = False

for bucket in response['Buckets']:
    print(bucket['Name'])
    if bucket['Name'] == export_bucket:
        export_bucket_exists = True

if not export_bucket_exists:
    s3.create_bucket(ACL="public-read-write", Bucket=export_bucket)
    
# Saving TF model to S3
tf.keras.models.save_model(model=model, filepath=export_path, save_format="tf")

response = s3.list_objects(Bucket='models')
for file in response['Contents']:
    print(file['Key'])

In [ ]:
imported_model = tf.keras.models.load_model(export_path)

In [ ]:
image_index = 1005

display_image(x_train, image_index)

In [ ]:
predict_number(imported_model, x_test, image_index)